In [29]:
import pickle
import json
import numpy as np
import requests
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed

In [2]:
with open("llama_answers.pkl", 'rb') as f:
    llama_answers = pickle.load(f)
    f.close()

with open("claude_answers.pkl", "rb") as f:
    claude_answers = pickle.load(f)
    f.close()

with open("compare_llama.pkl", "rb") as f:
    compare_llama = pickle.load(f)
    f.close()

with open("compare_claude.pkl", "rb") as f:
    compare_claude = pickle.load(f)
    f.close()

In [6]:
LLAMA_API = "https://stock.cmcts.ai/c-agent/api/v1/prediction/dd0f9487-4ab8-4dc6-8b16-b665e3121499"
CLAUDE_API = "https://stock.cmcts.ai/c-agent/api/v1/prediction/bb99269d-6f89-414e-8702-5d84c5f8e1a4"
GRADER_API = "https://stock.cmcts.ai/c-agent/api/v1/prediction/d6675cbc-88a6-4ca4-8f1a-cf660a60beb8"

In [49]:
from langfuse import Langfuse
langfuse = Langfuse(public_key="pk-lf-9c1bbc22-8180-4c1e-9d1c-4bedf12c56fc",
                    secret_key="sk-lf-d758beb3-be26-4e22-9f7b-847bd9142d2b",
                    host="https://us.cloud.langfuse.com",)

current_id  = langfuse.fetch_traces(limit=1).data[0].id
current_id

'f050e3cf-123d-48c4-9855-2e664d5e5550'

In [50]:
def query(payload, api):
    response = requests.post(api, json=payload)
    return response

In [52]:
def get_answers(questions, api, llm_node):
    global current_id
    answers = [] 
    for question in questions:
        payload = {"question": question}
        temp = query(payload, api)
        while True:
            data = langfuse.fetch_traces(limit=1).data[0]
            if current_id != data.id and (data.output is not None and not isinstance(data.output, str)):
                answers.append(data.output[llm_node]["messages"][0]["kwargs"]["content"])
                current_id = data.id
                print("Checkpoint: Got answer!")
                break
    return answers

In [53]:
def get_comparisons(questions, ground_truth_answers, llm_answers):
    global current_id
    comparisons = []
    for question, groundTruthAnswer, llmAnswer in zip(questions, ground_truth_answers, llm_answers):
        compare_question = f"""
        Question: {question}\n\n
        Ground-truth answer: {groundTruthAnswer}\n\n
        LLM answer: {llmAnswer}
        """
        payload = {"question": compare_question}
        temp = query(payload, GRADER_API)

        while True:
            data = langfuse.fetch_traces(limit=1).data[0]
            if current_id != data.id and (data.output is not None and not isinstance(data.output, str)):
                comparisons.append(data.output["tax_grader"]["messages"][0]["kwargs"]["content"])
                current_id = data.id
                print("Checkpoint: Got comparison")
                break
    return comparisons

In [54]:
def insert_subcol(score_df_data, comment_df_data, subcol, answers, data):
    new_score_df_data = {}
    for (top_level, sub_level), value in score_df_data.items():
        new_score_df_data[(top_level, sub_level)] = value
        if top_level == "Generation" and sub_level == "Llama":
            new_score_df_data[("Generation", subcol)] = data["generation_score"]
        
        if top_level == "Sources - Laws" and sub_level == "Llama":
            new_score_df_data[("Sources - Laws", subcol)] = data["sources_laws_score"]

        if top_level == "Brevity" and sub_level == "Llama":
            new_score_df_data[("Brevity", subcol)] = data["brevity_score"]

        if top_level == "Average score" and sub_level == "Llama":
            new_score_df_data[("Average score", subcol)] = data["average_score"]

    # Comment df
    new_comment_df_data = {}
    for (top_level, sub_level), value in comment_df_data.items():
        new_comment_df_data[(top_level, sub_level)] = value
        if top_level == "LLM anwser" and sub_level == "Llama":
            new_comment_df_data[("LLM anwser", subcol)] = answers
        if top_level == "Đánh giá" and sub_level == "Llama":
            new_comment_df_data[("Đánh giá", subcol)] = data["comment"]

    return new_score_df_data, new_comment_df_data

In [10]:
qa_df = pd.read_csv("data/tax_qa.csv")
qa_df.head()

,Câu hỏi,Câu trả lời sau rà soát
0,Người lao động ký hợp đồng lao động không thời...,'Căn cứ Điểm b.2 Khoản 1 Điều 25 Thông tư 111/...
1,Mẫu biểu quyết toán thuế TNCN của cty là mẫu n...,Tại điểm b tiết 9.9 Khoản 9 Phụ lục I ban hàn...
2,"""Bên em có trường hợp xuất Chứng từ khấu trừ t...",Đối với vướng mắc của Công ty về mẫu biểu tờ k...
3,NLĐ vừa có thời gian thử việc khấu trừ thuế TN...,- Điều 25 Thông tư số 111/2013/TT-BTC quy định...
4,Quyết toán thuế TNCN đối với cá nhân được điều...,Trường hợp cá nhân điều chuyển từ tổ chức cũ s...


In [11]:
questions = qa_df["Câu hỏi"].values
groundTruthAnswers = qa_df["Câu trả lời sau rà soát"].values

In [12]:
score_df = pd.read_excel("data/comparisons_v2.xlsx", sheet_name="Điểm", header=[0,1])
comment_df = pd.read_excel("data/comparisons_v2.xlsx", sheet_name="Đánh giá", header=[0,1])


### Claude sonnet 3.7

In [55]:
claude_sonnet_3_7_answers = get_answers(questions, CLAUDE_API, llm_node="tax_claude")

Checkpoint: Got answer!
Checkpoint: Got answer!
Checkpoint: Got answer!
Checkpoint: Got answer!
Checkpoint: Got answer!
Checkpoint: Got answer!
Checkpoint: Got answer!
Checkpoint: Got answer!
Checkpoint: Got answer!
Checkpoint: Got answer!


In [56]:
# Claude
claude_3_7_compare = get_comparisons(questions=questions, ground_truth_answers=groundTruthAnswers, llm_answers=claude_sonnet_3_7_answers)

Checkpoint: Got comparison
Checkpoint: Got comparison
Checkpoint: Got comparison
Checkpoint: Got comparison
Checkpoint: Got comparison
Checkpoint: Got comparison
Checkpoint: Got comparison
Checkpoint: Got comparison
Checkpoint: Got comparison
Checkpoint: Got comparison


In [58]:
average_score = 0
for compare in claude_3_7_compare:
    compare = json.loads(compare)
    generation_score = float(compare["generation_score"])
    sources_laws_score = float(compare["sources_laws_score"])
    brevity_score = float(compare["brevity_score"])
    average_score_per_cirtical = np.round((generation_score + sources_laws_score + brevity_score)/3, 2)
    average_score += average_score_per_cirtical
    print(generation_score, sources_laws_score, brevity_score, average_score_per_cirtical)
print(np.round(average_score/10, 2))


8.0 6.0 7.0 7.0
5.0 4.0 6.0 5.0
8.0 7.0 9.0 8.0
6.0 5.0 7.0 6.0
3.0 2.0 3.0 2.67
6.0 4.0 7.0 5.67
3.0 4.0 6.0 4.33
9.0 8.0 9.0 8.67
7.0 6.0 6.0 6.33
9.0 8.0 8.0 8.33
6.2


In [59]:
data = {"Question": [],
        "Ground-truth answer": [],
        "LLM answer": [],
        "Generation": [],
        "Generation score": [],
        "Sources - Laws": [],
        "Sources - Laws score": [],
        "Brevity": [],
        "Brevity score": [],
        "Comment": [],
        "Average score": [],
        "Strength Weakness": []}

for question, ground_truth_answer, llm_answer, comparison in zip(questions, groundTruthAnswers, claude_sonnet_3_7_answers, claude_3_7_compare):
    compare = json.loads(comparison)
    data["Question"].append(question)
    data["Ground-truth answer"].append(ground_truth_answer)
    data["LLM answer"].append(llm_answer)
    data["Generation"].append(compare["generation"])
    data["Sources - Laws"].append(compare["sources_laws"])
    data["Brevity"].append(compare["brevity"])
    data["Generation score"].append(float(compare["generation_score"]))
    data["Sources - Laws score"].append(float(compare["sources_laws_score"]))
    data["Brevity score"].append(float(compare["brevity_score"]))
    data["Comment"].append(compare["comment"])
    data["Average score"].append(np.round((data["Generation score"][-1]\
                                 + data["Sources - Laws score"][-1]\
                                 + data["Brevity score"][-1])/3, 2))
    data["Strength Weakness"].append(compare["strength_weakness"])

In [60]:
temp_df = pd.DataFrame(data)
temp_df

,Question,Ground-truth answer,LLM answer,Generation,Generation score,Sources - Laws,Sources - Laws score,Brevity,Brevity score,Comment,Average score,Strength Weakness
0,Người lao động ký hợp đồng lao động không thời...,'Căn cứ Điểm b.2 Khoản 1 Điều 25 Thông tư 111/...,Đối với trường hợp người lao động ký hợp đồng ...,Câu trả lời của mô hình LLM có độ chính xác kh...,8.0,Mô hình LLM đã trích dẫn một số văn bản pháp l...,6.0,Câu trả lời của mô hình LLM dài hơn so với câu...,7.0,Câu trả lời của mô hình LLM có độ chính xác tư...,7.00,Điểm mạnh:\n- Cung cấp thông tin chi tiết về c...
1,Mẫu biểu quyết toán thuế TNCN của cty là mẫu n...,Tại điểm b tiết 9.9 Khoản 9 Phụ lục I ban hàn...,"Theo quy định hiện hành, các mẫu biểu liên qua...",Câu trả lời của mô hình LLM có một số điểm chí...,5.0,Câu trả lời của mô hình LLM có đề cập đến một ...,4.0,Câu trả lời của mô hình LLM tương đối ngắn gọn...,6.0,"Đánh giá tổng thể, câu trả lời của mô hình LLM...",5.00,"Điểm mạnh:\n- Cấu trúc câu trả lời rõ ràng, dễ..."
2,"""Bên em có trường hợp xuất Chứng từ khấu trừ t...",Đối với vướng mắc của Công ty về mẫu biểu tờ k...,"Xin chào bạn,\n\nĐối với trường hợp xuất Chứng...",Câu trả lời của mô hình LLM có độ chính xác kh...,8.0,Mô hình LLM đã trích dẫn một số văn bản pháp l...,7.0,Câu trả lời của mô hình LLM có độ ngắn gọn và ...,9.0,"Nhìn chung, câu trả lời của mô hình LLM đã cun...",8.00,"Điểm mạnh:\n- Cấu trúc câu trả lời rõ ràng, dễ..."
3,NLĐ vừa có thời gian thử việc khấu trừ thuế TN...,- Điều 25 Thông tư số 111/2013/TT-BTC quy định...,Đối với trường hợp người lao động vừa có thời ...,Câu trả lời của mô hình LLM không hoàn toàn ch...,6.0,Mô hình có trích dẫn Thông tư số 80/2021/TT-BT...,5.0,Câu trả lời của mô hình khá ngắn gọn và súc tí...,7.0,Câu trả lời của mô hình LLM đã cung cấp một số...,6.00,"Điểm mạnh:\n- Câu trả lời ngắn gọn, dễ hiểu.\n..."
4,Quyết toán thuế TNCN đối với cá nhân được điều...,Trường hợp cá nhân điều chuyển từ tổ chức cũ s...,Đối với cá nhân được điều chuyển từ tổ chức cũ...,Câu trả lời của mô hình LLM có một số điểm khô...,3.0,Mô hình LLM đã trích dẫn Thông tư số 80/2021/T...,2.0,Câu trả lời của mô hình LLM dài hơn nhiều so v...,3.0,Câu trả lời của mô hình LLM có nhiều điểm khôn...,2.67,Điểm mạnh: Câu trả lời có cấu trúc rõ ràng và ...
5,Cá nhân trong năm vừa có thu nhập tính thuế th...,Căn cứ điểm d Khoản 6 Điều 8 Nghị định 126/202...,"Theo quy định hiện hành, cá nhân có đồng thời ...",Câu trả lời của mô hình LLM có một số điểm chí...,6.0,"Mô hình LLM đã cố gắng trích dẫn nguồn luật, n...",4.0,Câu trả lời của mô hình LLM có độ dài tương đố...,7.0,"Đánh giá tổng thể, câu trả lời của mô hình LLM...",5.67,Điểm mạnh:\n1. Cố gắng cung cấp thông tin về c...
6,"Mẫu số 02/QTT-TNCN/ TT80, trong trường hợp quy...",Người nộp thuế kê khai chỉ tiêu số thuế TNCN đ...,Theo quy định tại Thông tư số 80/2021/TT-BTC n...,Câu trả lời của mô hình LLM không chính xác so...,3.0,Mô hình có đề cập đến Thông tư số 80/2021/TT-B...,4.0,"Câu trả lời của mô hình khá ngắn gọn, nhưng do...",6.0,Câu trả lời của mô hình LLM có nhiều điểm khôn...,4.33,Điểm mạnh:\n- Câu trả lời ngắn gọn và dễ đọc.\...
7,Tôi đang kê khai quyết toán thuế nhưng phần kê...,Để khắc phục lỗi kê khai bảng kê (05-1/BK-TNCN...,Để khắc phục lỗi kê khai bảng kê (05-1/BK-TNCN...,Câu trả lời của mô hình LLM đã cung cấp thông ...,9.0,Câu trả lời của mô hình không trích dẫn cụ thể...,8.0,Câu trả lời của mô hình LLM truyền tải đầy đủ ...,9.0,"Câu trả lời của mô hình LLM rất tốt, cung cấp ...",8.67,Điểm mạnh:\n- Cung cấp hướng dẫn chính xác và ...
8,A/C cho e hỏi có bắt buộc phải điền mã số thuế...,Đơn vị chi trả thu nhập có trách nhiệm thực hi...,"Theo quy định hiện hành, việc điền mã số thuế ...",Câu trả lời của mô hình LLM có độ chính xác kh...,7.0,Mô hình LLM đã trích dẫn một số văn bản pháp l...,6.0,Câu trả lời của mô hình LLM dài hơn so với câu...,6.0,Câu trả lời của mô hình LLM đã cung cấp một số...,6.33,Điểm mạnh:\n- Đã nêu được yêu cầu bắt buộc về ...
9,"Trong năm 2023 tôi có làm việ

In [61]:
with pd.ExcelWriter("data/test/claude_sonnet_3_7.xlsx", engine="openpyxl") as writer:
    temp_df.to_excel(writer, sheet_name = "Prompt 1 Generator 0 Grader 0", header=True, index=False)

In [38]:
for index, comparison in enumerate(compare_claude_v2):
    compare = json.loads(comparison)
    generation = compare["generation"]
    sources_laws = compare["sources_laws"]
    brevity = compare["brevity"]
    generation_score = float(compare["generation_score"])
    sources_laws_score = float(compare["sources_laws_score"])
    brevity_score = float(compare["brevity_score"])
    comment = compare["comment"]
    strength_weakness = compare["strength_weakness"]

    score_df.loc[index, ("Generation", "Claude")] = str(generation_score) + "/10"
    score_df.loc[index, ("Sources - Laws", "Claude")] = str(sources_laws_score) + "/10"
    score_df.loc[index, ("Brevity", "Claude")] = str(brevity_score) + "/10"
    score_df.loc[index, ("Tổng điểm", "Claude")] = str(np.round((generation_score + sources_laws_score + brevity_score)/3, 2)) + "/10"
    comment_df.loc[index, "Câu trả lời sau rà soát"] = groundTruthAnswers[index]
    comment_df.loc[index, ("LLM anwser", "Claude")] = claude_answers[index]
    comment_df.loc[index, ("Đánh giá", "Claude")] = comment

### Claude sonnet 3.5

In [31]:
claude_sonnet_3_5_answers = [] 
for question in questions:
    payload = {"question": question}
    temp = query(payload, CLAUDE_API)
    while True:
        data = langfuse.fetch_traces(limit=1).data[0]
        if current_id != data.id and (data.output is not None and not isinstance(data.output, str)):
            claude_sonnet_3_5_answers.append(data.output["tax_claude"]["messages"][0]["kwargs"]["content"])
            current_id = data.id
            print("Checkpoint: Claude")
            break

Checkpoint: Claude
Checkpoint: Claude
Checkpoint: Claude
Checkpoint: Claude
Checkpoint: Claude
Checkpoint: Claude
Checkpoint: Claude
Checkpoint: Claude
Checkpoint: Claude
Checkpoint: Claude


In [32]:
# Claude
compare_claude= []
for question, groundTruthAnswer, llmAnswer in zip(questions, groundTruthAnswers, claude_sonnet_3_5_answers):
    compare_question = f"""
    Question: {question}\n\n
    Ground-truth answer: {groundTruthAnswer}\n\n
    LLM answer: {llmAnswer}
    """
    payload = {"question": compare_question}
    temp = query(payload, GRADER_API)

    # Fetch the most recent trace
    traces = langfuse.fetch_traces(limit=1)
    while True:
        data = langfuse.fetch_traces(limit=1).data[0]
        if current_id != data.id and (data.output is not None and not isinstance(data.output, str)):
            compare_claude.append(data.output["tax_grader"]["messages"][0]["kwargs"]["content"])
            current_id = data.id
            print("Checkpoint: Comparison")
            break

Checkpoint: Comparison
Checkpoint: Comparison
Checkpoint: Comparison
Checkpoint: Comparison
Checkpoint: Comparison
Checkpoint: Comparison
Checkpoint: Comparison
Checkpoint: Comparison
Checkpoint: Comparison
Checkpoint: Comparison


In [33]:
average_score = 0
for compare in compare_claude:
    compare = json.loads(compare)
    generation_score = float(compare["generation_score"])
    sources_laws_score = float(compare["sources_laws_score"])
    brevity_score = float(compare["brevity_score"])
    average_score_per_cirtical = np.round((generation_score + sources_laws_score + brevity_score)/3, 2)
    average_score += average_score_per_cirtical
    print(generation_score, sources_laws_score, brevity_score, average_score_per_cirtical)
print(np.round(average_score/10, 2))


1.0 0.0 0.0 0.33
3.0 2.0 3.0 2.67
1.0 0.0 1.0 0.67
1.0 0.0 2.0 1.0
2.0 1.0 3.0 2.0
2.0 1.0 3.0 2.0
2.0 3.0 3.0 2.67
1.0 2.0 3.0 2.0
1.0 0.0 2.0 1.0
0.0 0.0 2.0 0.67
1.5


In [38]:
compare_claude

['{"generation":"Câu trả lời của mô hình LLM không chính xác và không liên quan đến câu hỏi được đặt ra. Mô hình đã cung cấp thông tin về các mẫu biểu quyết toán thuế TNCN và đăng ký người phụ thuộc, trong khi câu hỏi yêu cầu hướng dẫn về cách tính thuế TNCN cho người lao động làm việc ngắn hạn và cách kê khai trong quyết toán thuế. Câu trả lời hoàn toàn không đề cập đến nội dung câu hỏi và không cung cấp thông tin hữu ích.","generation_score":"1","sources_laws":"Câu trả lời của mô hình không trích dẫn bất kỳ văn bản pháp luật nào liên quan đến câu hỏi. Trong khi đó, câu trả lời chuẩn đã trích dẫn chính xác Điểm b.2 Khoản 1 Điều 25 Thông tư 111/2013/TT-BTC, là văn bản pháp luật quan trọng để giải quyết vấn đề được hỏi.","sources_laws_score":"0","brevity":"Mặc dù câu trả lời của mô hình có độ dài vừa phải, nhưng nội dung hoàn toàn không liên quan đến câu hỏi. Vì vậy, không thể đánh giá tính ngắn gọn, súc tích của câu trả lời này.","brevity_score":"0","comment":"Câu trả lời của mô hình L

In [35]:
# Llama
compare_llama_v2 = []
for question, groundTruthAnswer, llmAnswer in zip(questions, groundTruthAnswers, llama_answers):
    compare_question = f"""
    Question: {question}\n\n
    Ground-truth answer: {groundTruthAnswer}\n\n
    LLM answer: {llmAnswer}
    """
    payload = {"question": compare_question}
    temp = query(payload, GRADER_API)

    # Fetch the most recent trace
    traces = langfuse.fetch_traces(limit=1)
    while True:
        data = langfuse.fetch_traces(limit=1).data[0]
        if current_id != data.id and (data.output is not None and not isinstance(data.output, str)):
            compare_llama_v2.append(data.output["tax_grader"]["messages"][0]["kwargs"]["content"])
            current_id = data.id
            print("Checkpoint: Comparison")
            break

Checkpoint: Comparison
Checkpoint: Comparison
Checkpoint: Comparison
Checkpoint: Comparison
Checkpoint: Comparison
Checkpoint: Comparison
Checkpoint: Comparison
Checkpoint: Comparison
Checkpoint: Comparison
Checkpoint: Comparison


In [39]:
for index, comparison in enumerate(compare_llama_v2):
    compare = json.loads(comparison)
    generation = compare["generation"]
    sources_laws = compare["sources_laws"]
    brevity = compare["brevity"]
    generation_score = float(compare["generation_score"])
    sources_laws_score = float(compare["sources_laws_score"])
    brevity_score = float(compare["brevity_score"])
    comment = compare["comment"]
    strength_weakness = compare["strength_weakness"]

    comparisons_score.loc[index, ("Generation", "Llama")] = str(generation_score) + "/10"
    comparisons_score.loc[index, ("Sources - Laws", "Llama")] = str(sources_laws_score) + "/10"
    comparisons_score.loc[index, ("Brevity", "Llama")] = str(brevity_score) + "/10"
    comparisons_score.loc[index, ("Tổng điểm", "Llama")] = str(np.round((generation_score + sources_laws_score + brevity_score)/3, 2)) + "/10"
    comparisons_comment.loc[index, "Câu trả lời sau rà soát"] = groundTruthAnswers[index]
    comparisons_comment.loc[index, ("LLM anwser", "Llama")] = llama_answers[index]
    comparisons_comment.loc[index, ("Đánh giá", "Llama")] = comment

In [31]:
cucthue_chatbot_df = pd.read_csv("data/cucthue_chatbot_answer.csv")
cucthue_chatbot_answers = cucthue_chatbot_df["cucthue_chatbot"].values

In [33]:
# Cucthue chatbot
compare_cucthue_chatbot = []
for question, groundTruthAnswer, llmAnswer in zip(questions, groundTruthAnswers, cucthue_chatbot_answers):
    compare_question = f"""
    Question: {question}\n\n
    Ground-truth answer: {groundTruthAnswer}\n\n
    LLM answer: {llmAnswer}
    """
    payload = {"question": compare_question}
    temp = query(payload, GRADER_API)

    # Fetch the most recent trace
    traces = langfuse.fetch_traces(limit=1)
    while True:
        data = langfuse.fetch_traces(limit=1).data[0]
        if current_id != data.id and (data.output is not None and not isinstance(data.output, str)):
            compare_cucthue_chatbot.append(data.output["tax_grader"]["messages"][0]["kwargs"]["content"])
            current_id = data.id
            print("Checkpoint: Comparison")
            break

Checkpoint: Comparison
Checkpoint: Comparison
Checkpoint: Comparison
Checkpoint: Comparison
Checkpoint: Comparison
Checkpoint: Comparison
Checkpoint: Comparison
Checkpoint: Comparison
Checkpoint: Comparison
Checkpoint: Comparison


In [40]:
for index, comparison in enumerate(compare_cucthue_chatbot):
    compare = json.loads(comparison)
    generation = compare["generation"]
    sources_laws = compare["sources_laws"]
    brevity = compare["brevity"]
    generation_score = float(compare["generation_score"])
    sources_laws_score = float(compare["sources_laws_score"])
    brevity_score = float(compare["brevity_score"])
    comment = compare["comment"]
    strength_weakness = compare["strength_weakness"]

    comparisons_score.loc[index, ("Generation", "Chatbot cục thuế")] = str(generation_score) + "/10"
    comparisons_score.loc[index, ("Sources - Laws", "Chatbot cục thuế")] = str(sources_laws_score) + "/10"
    comparisons_score.loc[index, ("Brevity", "Chatbot cục thuế")] = str(brevity_score) + "/10"
    comparisons_score.loc[index, ("Tổng điểm", "Chatbot cục thuế")] = str(np.round((generation_score + sources_laws_score + brevity_score)/3, 2)) + "/10"
    comparisons_comment.loc[index, "Câu trả lời sau rà soát"] = groundTruthAnswers[index]
    comparisons_comment.loc[index, ("LLM anwser", "Chatbot cục thuế")] = cucthue_chatbot_answers[index]
    comparisons_comment.loc[index, ("Đánh giá", "Chatbot cục thuế")] = comment

/tmp/ipykernel_336966/963092113.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '8.0/10' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  comparisons_score.loc[index, ("Generation", "Chatbot cục thuế")] = str(generation_score) + "/10"
/tmp/ipykernel_336966/963092113.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '7.0/10' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  comparisons_score.loc[index, ("Sources - Laws", "Chatbot cục thuế")] = str(sources_laws_score) + "/10"
/tmp/ipykernel_336966/963092113.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '6.0/10' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
 

In [62]:
with pd.ExcelWriter("data/comparisons.xlsx", engine="openpyxl") as writer:
    comparisons_score.to_excel(writer, sheet_name="Điểm")
    comparisons_comment.to_excel(writer, sheet_name="Đánh giá")

In [63]:
comparisons_score

Tiêu chí\n\n\n\nCâu hỏi Generation          \
                                  Unnamed: 0_level_1     Claude   Llama   
0  Người lao động ký hợp đồng lao động không thời...     6.0/10  6.0/10   
1  Mẫu biểu quyết toán thuế TNCN của cty là mẫu n...     5.0/10  6.0/10   
2  "Bên em có trường hợp xuất Chứng từ khấu trừ t...     7.0/10  5.0/10   
3  NLĐ vừa có thời gian thử việc khấu trừ thuế TN...     7.0/10  3.0/10   
4  Quyết toán thuế TNCN đối với cá nhân được điều...     6.0/10  5.0/10   
5  Cá nhân trong năm vừa có thu nhập tính thuế th...     3.0/10  7.0/10   
6  Mẫu số 02/QTT-TNCN/ TT80, trong trường hợp quy...     4.0/10  3.0/10   
7  Tôi đang kê khai quyết toán thuế nhưng phần kê...     9.0/10  9.5/10   
8  A/C cho e hỏi có bắt buộc phải điền mã số thuế...     8.0/10  6.0/10   
9  Trong năm 2023 tôi có làm việc tại Nha Trang, ...     6.0/10  5.0/10   

                   Sources - Laws                          Brevity          \
  Chatbot cục thuế         Claude   Llama Chatbot cục thuế  Claude   Llama   
0           8.0/10         5.0/10  5.0/10           7.0/10  6.0/10  5.0/10   
1           7.0/10         4.0/10  1.0/10           6.0/10  6.0/10  6.0/10   
2           6.0/10         6.0/10  4.0/10           5.0/10  7.0/10  6.0/10   
3           8.0/10         6.0/10  2.0/10           7.0/10  8.0/10  5.0/10   
4           5.0/10         5.0/10  4.0/10           3.0/10  4.0/10  2.0/10   
5           3.0/10         2.0/10  6.0/10           2.0/10  4.0/10  8.0/10   
6           3.0/10         6.0/10  2.0/10           1.0/10  6.0/10  4.0/10   
7           6.0/10         7.0/10  8.0/10           8.0/10  8.0/10  9.0/10   
8           5.0/10         7.0/10  4.0/10           3.0/10  6.0/10  5.0/10   
9           9.0/10         7.0/10  6.0/10           9.0/10  4.0/10  4.0/10   

                   Tổng điểm                            
  Chatbot cục thuế    Claude    Llama Chatbot cục thuế  
0           6.0/10   5.67/10  5.33/10           7.0/10  
1           7.0/10    5.0/10  4.33/10          6.67/10  
2           7.0/10   6.67/10   5.0/10           6.0/10  
3           8.0/10    7.0/10  3.33/10          7.67/10  
4           2.0/10    5.0/10  3.67/10          3.33/10  
5           4.0/10    3.0/10   7.0/10           3.0/10  
6           4.0/10   5.33/10   3.0/10          2.67/10  
7           5.0/10    8.0/10  8.83/10          6.33/10  
8           4.0/10    7.0/10   5.0/10           4.0/10  
9           7.0/10   5.67/10   5.0/10          8.33/10

In [8]:
with pd.ExcelWriter("data/comparisons.xlsx", engine="openpyxl") as writer:
    score_df.to_excel(writer, sheet_name="Điểm")
    comment_df.to_excel(writer, sheet_name="Đánh giá")

In [71]:
data = {'generation': {},
        'generation_score': {},
        'sources_laws': {},
        'sources_laws_score': {},
        'brevity': {},
        'brevity_score': {},
        'comment': {},
        'strength_weakness': {},
        'average_score': {}}
for index, compare in enumerate(compare_claude_v2):
    compare = json.loads(compare)
    
    data['generation'].update({index: compare['generation']})
    data['generation_score'].update({index:float(compare['generation_score'])})
    data['sources_laws'].update({index: compare['sources_laws']})
    data['sources_laws_score'].update({index: float(compare['sources_laws_score'])})
    data['brevity'].update({index: compare['brevity']})
    data['brevity_score'].update({index: float(compare['brevity_score'])})
    data['comment'].update({index: compare['comment']})
    data['strength_weakness'].update({index: compare['strength_weakness']})
    data['average_score'].update({index: np.round((float(compare['generation_score']) + float(compare['sources_laws_score']) + float(compare['brevity_score'])) / 3, 2)})

In [72]:
score_df_data = score_df.to_dict()
comment_df_data = comment_df.to_dict()

In [73]:
answers = {i: answer for i, answer in enumerate(claude_sonnet_3_7_answers)}
new_score_df_data, new_comment_df_data = insert_subcol(score_df_data=score_df_data,
              comment_df_data=comment_df_data,
              subcol="Claude 3.7",
              answers=answers,
              data=data)

In [74]:
new_score_df = pd.DataFrame(new_score_df_data)
new_comment_df = pd.DataFrame(new_comment_df_data)

In [75]:
with pd.ExcelWriter("data/comparisons.xlsx", engine="openpyxl") as writer:
    new_score_df.to_excel(writer, sheet_name="Điểm")
    new_comment_df.to_excel(writer, sheet_name="Đánh giá")